# IBM DATA SCIENCE - CAPSTONE PROJECT
---------------------------
This Jupyter Notebook will be used for the final project of the IBM data science certification.

In [1]:
# Importing Libraries
import pandas as pd
import numpy as np

In [2]:
# 
print("Hello Capstone Project Course!")

NameError: name 'prin' is not defined